<a href="https://colab.research.google.com/github/shinnakayama/GLMM-course/blob/master/convert_to_DOTA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
import numpy as np
import pandas as pd
import json
import os
from PIL import Image, ImageDraw
import math
from google.cloud import storage

Auth Google Cloud

In [ ]:
!gcloud config set project planetport

from google.colab import auth
auth.authenticate_user()

Download gcsfuse and mount Google Cloud Storage bucket

In [45]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

# create a folder to mount
temp_folder_name = 'temp/'
if not os.path.exists(temp_folder_name):
    os.mkdir(temp_folder_name)

# mount
my_bucket = 'planet_imagery'
!gcsfuse --implicit-dirs {my_bucket} {temp_folder_name}

gcsfuse is already the newest version (1.2.0).
0 upgraded, 0 newly installed, 0 to remove and 32 not upgraded.
{"time":"06/10/2023 01:34:23.285636","severity":"INFO","msg":"Start gcsfuse/1.2.0 (Go version go1.21.0) for app \"\" using mount point: /content/temp\n"}


Load data and add GSD

In [32]:
prefix = 'tema_2022'

# output of Label Studio
labels = pd.read_csv('temp/' + prefix+ '/labels.csv')
labels['image_id'] = ['_'.join(x.split('/')[-1][9:].split('_')[:3]) for x in labels['image']]


# from "chop_images.ipynb"
image_info = pd.read_csv('temp/' + prefix + '/image_info.csv')
image_info['image_id'] = ['_'.join(x.split('_')[:3]) for x in image_info['image']]


# add gsd to label
labels = pd.merge(labels, image_info[['image_id', 'gsd']], how='left', on='image_id')

Save labels for each image as txt

In [47]:
if not os.path.exists('temp/' + prefix + '/label'):
    os.makedirs('temp/' + prefix + '/label')

for _, row in labels.iterrows():

    foo = row.label

    # skip images without any objects
    if foo is np.nan:
        continue

    df = pd.DataFrame(json.loads(foo))

    # convert to pixel and radian
    df['x'] = df['x']/100 * df['original_width']
    df['y'] = df['y']/100 * df['original_height']
    df['width'] = df['width']/100 * df['original_width']
    df['height'] = df['height']/100 * df['original_height']
    df['rotation'] = [math.radians(x) for x in df['rotation']]

    annotation = 'imagesource:PSScene\ngsd:' + str(row['gsd']) + '\n'

    for _, inner_row in df.iterrows():
        x1 = inner_row['x']
        y1 = inner_row['y']
        width = inner_row['width']
        height = inner_row['height']
        z_radian = inner_row['rotation']

        # rotate
        x2 = width * math.cos(z_radian) - 0 * math.sin(z_radian) + x1
        y2 = width * math.sin(z_radian) + 0 * math.cos(z_radian) + y1

        x3 = width * math.cos(z_radian) - height * math.sin(z_radian) + x1
        y3 = width * math.sin(z_radian) + height * math.cos(z_radian) + y1

        x4 = 0 * math.cos(z_radian) - height * math.sin(z_radian) + x1
        y4 = 0 * math.sin(z_radian) + height * math.cos(z_radian) + y1

        rotated_corners = [(x1, y1), (x2, y2), (x3, y3), (x4, y4)]

        # label
        label = inner_row['rectanglelabels'][0]
        new_label = '_'.join(label.split('_')[:-1])  # remove 'easy/difficult'

        # difficulty
        difficulty = 0 if label.split('_')[-1] == 'easy' else 1

        # add
        annotation += ' '.join([f"{x:.3f} {y:.3f}" for x, y in rotated_corners])
        annotation += ' ' + new_label + ' ' + str(difficulty) + '\n'

    # save
    image_file = row.image.split('/')[-1][9:]
    text_file = image_file.replace('jpg', 'txt')
    file_path = 'temp/' + prefix + '/label/' + text_file
    with open(file_path, 'w') as file:
        file.write(annotation)